# Fast Tutorial 1 - Algorithm Aggregation

In this notebook, I will explain how to use the *embedded_voting* package in the context of epistemic social choice and algorithms aggregations.

In general algorithm aggregation rules (Average, Median, Likelihood maximization) you need diversity among the different algorithms. However, in the real world, it is not rare to have a large group of very correlated algorithms, which are trained on the same datasets, or which have the same structure.

With our method, you can take advantage of the dependencies between the rules instead of suffering from them. In this first notebook, I will just explain how to use our method. In the following method, I will show comparisons between our method and other methods.

First of all, you need to import the package:

In [1]:
import embedded_voting as ev

## Generator to simulate algorithm results

Then, if you want to aggregate algorithms' outputs, you need to know the outputs of these algorithms. In this notebook, I will use a score generator that simulates a set of algorithms with dependencies.

In the following cell, I create a set of algorithms with $25$ algorithms in the first group, $7$ in the second group and $3$ isolated algorithms.

In [2]:
groups_sizes = [25, 7, 1, 1, 1]
features = [[1, 0, 0, 1], [0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0]]

generator = ev.GroupedMixGenerator(groups_sizes, features)
generator.set_group_noise(8)
generator.set_independent_noise(0.5)

_, scores = generator.sample_scores(20)
print(scores.shape)

(35, 20)


The last command generates a matrix of scores that contain the outputs given by the algorithms to 20 inputs. If you use this method, you can provide the score matrix by putting your algorithms' results in a matrix of shape $n_{voters} \times n_{candidates}$.


## Find the best alternative

Now, you can simply **create an *Elector* object** with the following line:

In [3]:
my_elector = ev.Elector()

The following cell show how to run a "election":

In [4]:
results = my_elector(scores)

Then you can get the results like this:

In [5]:
print("Ranking :",results.ranking_)
print("Winner :",results.winner_)

Ranking : [8, 5, 0, 16, 3, 17, 13, 19, 18, 9, 4, 11, 6, 12, 2, 7, 14, 1, 15, 10]
Winner : 8


You will probably keep using the same *Elector* for other elections with the same algorithms, like in the following cell:

In [6]:
for i in range(10):
    _, scores = generator.sample_scores(20)
    print("Winner %i : %i"%(i+1, my_elector(scores).winner_))

Winner 1 : 10
Winner 2 : 4
Winner 3 : 14
Winner 4 : 9
Winner 5 : 5
Winner 6 : 13
Winner 7 : 10
Winner 8 : 11
Winner 9 : 16
Winner 10 : 11


During each election, the *Elector* saves the scores given by the algorithms to know them better.

Every now and then, you can retrain your *Elector* with these newest data. We advise to do it often where there is not a lot of training data and once you have done enough elections (typically, when you have shown as many candidates than you have algorithms), you don't need to do it a lot.

To train your *Elector* on the newest data, do the following:

In [7]:
my_elector.retrain()

You can also train it before an election using the data from the election by doing this:

In [8]:
results = my_elector(scores, train=True)

## Fine-tune the aggregation rule

If you want to go further, you can change some aspects of the aggregation rule.

The first thing that you may want to change is the aggregation rule itself. The default one is *FastNash*, but you can try *FastLog*, *FastSum* or *FastMin*, which can give different results.

We advise to use *FastNash*, which shows stronger theoretical and experimental results.

In [9]:
my_elector_log = ev.Elector(rule=ev.FastLog())
my_elector_sum = ev.Elector(rule=ev.FastSum())
my_elector_min = ev.Elector(rule=ev.FastMin())
print("FastNash:",my_elector(scores).ranking_)
print("FastLog:",my_elector_log(scores).ranking_)
print("FastSum:",my_elector_sum(scores).ranking_)
print("FastMin:",my_elector_min(scores).ranking_)

FastNash: [11, 10, 16, 4, 1, 7, 2, 15, 8, 3, 12, 13, 14, 18, 5, 19, 9, 17, 6, 0]
FastLog: [11, 10, 16, 4, 1, 7, 2, 15, 8, 3, 12, 13, 14, 18, 5, 19, 9, 17, 6, 0]
FastSum: [11, 16, 10, 15, 4, 1, 7, 2, 8, 13, 3, 12, 14, 18, 17, 5, 9, 19, 6, 0]
FastMin: [10, 4, 1, 11, 2, 7, 16, 15, 19, 5, 3, 8, 12, 13, 18, 9, 14, 0, 6, 17]


You can also use the average rule:

In [10]:
my_elector_avg = ev.Elector(rule=ev.SumScores())
results = my_elector_avg(scores)
print(my_elector_avg(scores).ranking_)

[11, 16, 10, 15, 1, 4, 7, 2, 8, 13, 3, 12, 14, 18, 17, 5, 9, 6, 19, 0]


You can also change the transformation of scores. The default one is the following :

$$
f(s) = \sqrt{\frac{s}{\left || s \right ||}}
$$

But you can put any rule you want, like the identity function $f(s) = s$ if you want. In general, if you use a coherent score transformation, it will not change a lot the results.

In [11]:
my_elector_id = ev.Elector(rule=ev.FastNash(function=lambda x:x))
print(my_elector_id(scores).ranking_)

[10, 11, 16, 4, 1, 7, 2, 15, 8, 3, 12, 13, 14, 18, 5, 19, 9, 17, 6, 0]
